In [1]:
!pip install classy-classification

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 26.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 16.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 37.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.2/280.2 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 22.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.7/33.7 MB 26.2 MB/s eta 0:00:0000:0100:01
  Using cached threadpoolctl-3.1.0-py3-none-any.whl (14 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.3/24.3 MB 29.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 48.1 MB/s eta 0:00:00
  Using cached sentencepiece-0.1.97-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.3 MB)
     ━━━━━━━━━━━━━━━

In [89]:
import os

import pandas as pd
import spacy
import classy_classification

In [ ]:
df = pd.read_csv('First Funnel - Train Examples.csv')
df.set_index('Question type', inplace=True)
df.sort_index(ignore_index=True)

In [21]:
train_dict = {}
for cat_i, df_cat_i in df.groupby('Question type'):
    train_dict[cat_i] = df_cat_i['Input text'].to_list()

persist it to json file

In [94]:
output_path = os.path.join('..', '..', 'setup_data', 'first_filter_train_examples.json')

In [96]:
import json

with open(output_path, 'w') as fp:
    json.dump(train_dict, fp, sort_keys=True, indent=4)

## Spacy's middle model

In [82]:

data = {
    "furniture": ["This text is about chairs.",
               "Couches, benches and televisions.",
               "I really need to get a new sofa."],
    "kitchen": ["There also exist things like fridges.",
                "I hope to be getting a new stove today.",
                "Do you also have some ovens."]
}

nlp = spacy.load("en_core_web_md")
nlp.add_pipe(
    "text_categorizer",
    config={
        "data": data,
        "model": "spacy",
        "cat_type": "multi-label",
    }
)

print(nlp("I am looking for kitchen appliances.")._.cats)

['furniture' 'kitchen'] [0.46483467 0.53516533]
{'furniture': 0.4648346710750303, 'kitchen': 0.5351653289249698}


Train few-shot categorizer

In [83]:
nlp = spacy.load("en_core_web_lg")
nlp.add_pipe(
    "text_categorizer",
    config={
        "data": train_dict,
        "model": "spacy"
    }
)

In [84]:
print(nlp("Hey, i am interested in a movie recomendation that is as good as The Goonies and Toy Story")._.cats)

['Conversation' 'Factual Question/Embedding/Crowdsourcing'
 'Media Question' 'Recommendation Questions'] [0.16959642 0.28293407 0.12628045 0.42118907]
{'Conversation': 0.16959641509573653, 'Factual Question/Embedding/Crowdsourcing': 0.2829340679385176, 'Media Question': 0.1262804474196118, 'Recommendation Questions': 0.42118906954613405}


In [85]:

print(nlp("Hey how is it going")._.cats)

['Conversation' 'Factual Question/Embedding/Crowdsourcing'
 'Media Question' 'Recommendation Questions'] [0.65992502 0.22482746 0.08236993 0.03287759]
{'Conversation': 0.6599250210440973, 'Factual Question/Embedding/Crowdsourcing': 0.22482746057270114, 'Media Question': 0.08236992624372713, 'Recommendation Questions': 0.032877592139474277}


In [101]:
print(nlp("hey would you be so kind to display picture Ali G")._.cats)

['Conversation' 'Factual Question/Embedding/Crowdsourcing'
 'Media Question' 'Recommendation Questions'] [0.06518361 0.04043501 0.87013233 0.02424905]
{'Conversation': 0.06518361222514099, 'Factual Question/Embedding/Crowdsourcing': 0.04043501102190481, 'Media Question': 0.8701323267631766, 'Recommendation Questions': 0.024249049989777383}


In [87]:
print(nlp("I bet you have absolutely no idea who is the voice of Tanjirou in Kimetsu no Yaiba")._.cats)

['Conversation' 'Factual Question/Embedding/Crowdsourcing'
 'Media Question' 'Recommendation Questions'] [0.27453749 0.61499526 0.03647736 0.0739899 ]
{'Conversation': 0.2745374861189631, 'Factual Question/Embedding/Crowdsourcing': 0.6149952554559426, 'Media Question': 0.036477362756375656, 'Recommendation Questions': 0.0739898956687187}


it's not that bad, but it is much better with the sentence-transformer

## Using sentence-transformer

In [88]:
import spacy
import classy_classification

data = {
    "furniture": ["This text is about chairs.",
               "Couches, benches and televisions.",
               "I really need to get a new sofa."],
    "kitchen": ["There also exist things like fridges.",
                "I hope to be getting a new stove today.",
                "Do you also have some ovens."]
}

nlp = spacy.blank("en")
nlp.add_pipe(
    "text_categorizer",
    config={
        "data": data,
        "model": "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
        "device": "gpu",
        "cat_type": "multi-label"
    }
)

print(nlp("I am looking for kitchen appliances.")._.cats)

Model found at: /home/juanbermeo/.cache/torch/sentence_transformers/sentence-transformers_paraphrase-multilingual-MiniLM-L12-v2/quantized_true.onnx
['furniture' 'kitchen'] [0.14006391 0.85993609]
{'furniture': 0.1400639084824182, 'kitchen': 0.8599360915175817}


In [99]:
nlp("I am looking for kitchen appliances.")

['Conversation' 'Factual Question/Embedding/Crowdsourcing'
 'Media Question' 'Recommendation Questions'] [0.06383076 0.6337598  0.1241994  0.17821004]


I am looking for kitchen appliances.

### Let's try with our few examples of question types

Train few-shot categorizer

In [97]:
nlp = spacy.blank("en")
nlp.add_pipe(
    "text_categorizer",
    config={
        "data": train_dict,
        "model": "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
        #"device": "gpu",
        "cat_type": "multi-label"
    }
)

Model found at: /home/juanbermeo/.cache/torch/sentence_transformers/sentence-transformers_paraphrase-multilingual-MiniLM-L12-v2/quantized_true.onnx


In [98]:
nlp("Hey, i am interested in a movie recomendation that is as good as The Goonies and Toy Story")

['Conversation' 'Factual Question/Embedding/Crowdsourcing'
 'Media Question' 'Recommendation Questions'] [0.05190779 0.18905881 0.03160923 0.72742417]


Hey, i am interested in a movie recomendation that is as good as The Goonies and Toy Story

In [61]:
print(nlp("Hey, i am interested in a movie recomendation that is as good as The Goonies and Toy Story")._.cats)

['Conversation' 'Factual Question/Embedding/Crowdsourcing'
 'Media Question' 'Recommendation Questions'] [0.04603117 0.19062184 0.03005194 0.73329505]
{'Conversation': 0.046031172799330396, 'Factual Question/Embedding/Crowdsourcing': 0.1906218359115138, 'Media Question': 0.03005194240328927, 'Recommendation Questions': 0.7332950488858668}


In [62]:

print(nlp("Hey how is it going")._.cats)

['Conversation' 'Factual Question/Embedding/Crowdsourcing'
 'Media Question' 'Recommendation Questions'] [0.93933056 0.02474838 0.02455568 0.01136538]
{'Conversation': 0.9393305623784225, 'Factual Question/Embedding/Crowdsourcing': 0.02474837841420033, 'Media Question': 0.024555676041556174, 'Recommendation Questions': 0.011365383165821152}


In [63]:
print(nlp("would you be so kind to show me a picture Ali G")._.cats)

['Conversation' 'Factual Question/Embedding/Crowdsourcing'
 'Media Question' 'Recommendation Questions'] [0.06040599 0.04568918 0.87636338 0.01754145]
{'Conversation': 0.060405993110324525, 'Factual Question/Embedding/Crowdsourcing': 0.045689179797099216, 'Media Question': 0.876363378829692, 'Recommendation Questions': 0.01754144826288433}


In [64]:
print(nlp("I bet you have absolutely no idea who is the voice of Tanjirou in  Kimetsu no Yaiba")._.cats)


['Conversation' 'Factual Question/Embedding/Crowdsourcing'
 'Media Question' 'Recommendation Questions'] [0.01774493 0.96483208 0.01232319 0.0050998 ]
{'Conversation': 0.017744926597209294, 'Factual Question/Embedding/Crowdsourcing': 0.9648320791035818, 'Media Question': 0.012323194924284436, 'Recommendation Questions': 0.005099799374924374}
